In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
import re
import json
import numpy as np
from nltk import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import pickle

from config import *
stemmer = LancasterStemmer()
pickle.dump(stemmer, open(saved_stemmer, 'wb'))

def prepare_data():
    with open(dataset_path) as file:
        data = json.load(file)

    try:
        with open(saved_params, "rb") as f:
            words, labels, training, output = pickle.load(f)
            print(words, labels, training, output)
            return training, output
    except:
        words = []
        labels = []
        docs_x = []
        docs_y = []

        for intent in data["intents"]:
            for pattern in intent["patterns"]:
                wrds = word_tokenize(pattern)
                words.extend(wrds)
                docs_x.append(wrds)
                docs_y.append(intent["tag"])

            if intent["tag"] not in labels:
                labels.append(intent["tag"])

        words = [stemmer.stem(w.lower()) for w in words if w != "?"]
        words = sorted(list(set(words)))

        labels = sorted(labels)

        training = []
        output = []

        out_empty = [0 for _ in range(len(labels))]

        for x, doc in enumerate(docs_x):
            bag = []

            wrds = [stemmer.stem(w.lower()) for w in doc]

            for w in words:
                if w in wrds:
                    bag.append(1)
                else:
                    bag.append(0)

            output_row = out_empty[:]
            output_row[labels.index(docs_y[x])] = 1

            training.append(bag)
            output.append(output_row)

        training = np.array(training)
        output = np.array(output)

        with open(saved_params, "wb") as f:
            pickle.dump((words, labels, training, output), f)

        
        return training, output



In [3]:
# defining data parsing functions

def find_dataset(path):
    with open(path) as file:
        data = json.load(file)
        return data

training_sentences = []
training_labels = []
labels = []
responses = []

def prep_data(path):
    """
    :returns: training_sentences, training_labels, responses(list), labels, num_classes
    """
    data = find_dataset(path=path)

    for intent in data['intents']:
        for pattern in intent['patterns']:
            training_sentences.append(pattern)
            training_labels.append(intent['tag'])
        responses.append(intent['responses'])
        
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    num_classes = len(labels)

    return training_sentences, training_labels, responses, labels, num_classes, data
training_sentences, training_labels, responses, labels, num_classes, data = prep_data('data/data.json')

In [4]:
training, output = prepare_data()

['a', 'abl', 'again', 'am', 'amaz', 'ar', 'awesom', 'be', 'bye', 'cal', 'can', 'chil', 'cool', 'crack', 'day', 'did', 'do', 'doing', 'down', 'ep', 'fre', 'from', 'giv', 'glad', 'good', 'gre', 'hah', 'happy', 'hav', 'heh', 'hello', 'hey', 'hi', 'hip', 'hobby', 'hop', 'how', 'howdy', 'humid', 'i', 'in', 'is', 'it', 'jazz', 'jok', 'jot', 'know', 'lat', 'latest', 'laugh', 'leav', 'lik', 'lmao', 'lof', 'lol', 'mak', 'me', 'mus', 'nam', 'new', 'not', 'on', 'play', 'provid', 'rain', 'rec', 'sdf', 'see', 'should', 'so', 'som', 'song', 'sup', 'talk', 'tel', 'temp', 'thank', 'that', 'the', 'ther', 'thi', 'tim', 'to', 'today', 'up', 'wak', 'was', 'weath', 'what', 'when', 'wher', 'who', 'whom', 'wil', 'work', 'writ', 'x', 'xyz', 'y', 'ye', 'yo', 'you', 'youtub', 'z'] ['blank', 'chat1', 'comment', 'cud', 'goodbye', 'greeting', 'happiness', 'hru', 'info', 'joke', 'jotnote', 'lofi', 'memorize', 'name', 'news', 'random', 'thank', 'weather', 'wrud', 'youtube'] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 

In [5]:
class NLU(nn.Module):
    
    def __init__(self):
        super(NLU, self).__init__()
        self.inp = nn.Linear(len(training[0]), 16)
        self.h = nn.Linear(16, 8)
        self.h2 = nn.Linear(8, 8)
        self.o = nn.Linear(8, len(output[0]))
        
    def forward(self, x):
        x = F.elu(self.inp(x))
        x = F.elu(self.h(x))
        x = F.elu(self.h2(x))
        x = self.o(x)
        return F.sigmoid(x)
        
nlu = NLU()
print(nlu)

NLU(
  (inp): Linear(in_features=104, out_features=16, bias=True)
  (h): Linear(in_features=16, out_features=8, bias=True)
  (h2): Linear(in_features=8, out_features=8, bias=True)
  (o): Linear(in_features=8, out_features=20, bias=True)
)


In [6]:
params = list(nlu.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

8
torch.Size([16, 104])


In [7]:
np.array(output).shape

(105, 20)

In [8]:
training.shape

(105, 104)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(nlu.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    ts = 0.0
    for i, data in enumerate(training, 0):
        # get the inputs; data is a list of [inputs, labels]
        x, y = torch.Tensor(data), torch.Tensor(output[i])

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        y_hat = nlu(x)
        y_hat = y_hat.reshape(1, 20)
        y = y.reshape(1, 20)

        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        
        
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 5:.3f}')
            running_loss = 0.0

print('Finished Training')

c:\users\romoni gogoi\appdata\local\programs\python\python36\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,    10] loss: 6.022
[1,    20] loss: 6.058
[1,    30] loss: 5.945
[1,    40] loss: 5.944
[1,    50] loss: 6.039
[1,    60] loss: 6.087
[1,    70] loss: 6.104
[1,    80] loss: 5.969
[1,    90] loss: 6.003
[1,   100] loss: 5.983
Finished Training


In [11]:
y.shape

torch.Size([1, 20])

In [12]:
PATH = 'model.pth'
torch.save(nlu.state_dict(), PATH)

In [13]:
net = NLU()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
import random
from config import *


data = json.load(open(dataset_path))
stemmer = pickle.load(open(saved_stemmer, 'rb'))

words, labels, _, _ = pickle.load(open(saved_params, 'rb'))



def bag_of_words(s, words):
    """
    Creating a bag of words to store how many times a word matches with the
    data and inputs 1 or 0. For example, for a text in intent["patterns"] if
    the word in text appears in the data, then in the bag, we will insert 1,
    for the next word, if it doesn't exists, then we will enter 0 to the bag.
    Finally we will match all the text for the number of times 1 occurs.
    The pattern with the max number of 1 will be taken as the correct match,
    and we will output a random response from intent["responses"] followed
    by proper indices

    """

    bag = [0 for _ in range(len(words))]

    s_words = word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)


def argmax(array):
  index, value = 0, array[0]
  for i,v in enumerate(array):
    if v > value:
      index, value = i,v
  return index


def chat():
    """
    Final function that takes no arguments. In the starting of this
    function, we are creating a table of our database. In the function method
    create_table(bool) it checks for if the table exists, then passes
    the method, if not then creates one.

    In the results = model.predict(), we are passing the input from user
    to match with the training data. The second arg that we are passing is a list
    which is sorted with no duplicate words.

    The most important statements are inside the for loop here. for tg in data["intents"]
    if the matched words and patterns fall in a particular tag, then grab that tag,
    and return any random response from the responses tag of the trained json.


    """

    print("Start talking with the bot (type quit to stop)!")
    inp = input("You: ")
    if inp.lower() == "quit":
        quit()

    results = net(torch.Tensor([bag_of_words(inp, words)]))
    results_index = np.argmax(results.detach().numpy())
    print(results_index)
    tag = labels[results_index]
    print(tag)
    print(labels)

    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']

    return random.choice(responses)
chat()

Start talking with the bot (type quit to stop)!


In [ ]:
#FROM SCRATCH